#Costas

In [12]:
from lxml import html
import requests
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
import re


In [13]:
#Mongo
import pymongo
from pymongo import MongoClient

client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.AEMET

In [14]:
# Añado un dataframe con los meses para la fecha
global meses 
df_meses = pd.DataFrame()
df_meses["es"] = ["enero","febrero","marzo","abril","mayo","junio","julio","agosto"
                ,"septiembre","octubre","noviembre","diciembre"]
df_meses["en"] = ["january","february", "march", "april","may","june","july","august"
                ,"september","october","november", "december"]
df_meses["num"] = ["01","02", "03", "04","05","06","07","08","09","10","11", "12"]

def traduceMes(mes,lang_origen,lang_destino):
    for index, row in meses.iterrows():
        if(mes.upper().find(row[lang_origen].upper())==0):
            mes = mes.upper().replace(row[lang_origen].upper(),row[lang_destino])
            return mes

def getFechaTime(txt):
    arrayFecha = txt.split(" ")
    mes =  df_meses.ix[df_meses.es==arrayFecha[1].lower()].en
    nueva_fecha =  arrayFecha[0] + mes.values[0] + arrayFecha[2]
    fecha =datetime.strptime(nueva_fecha,'%d%B%Y')
    return fecha



In [15]:
url = "http://www.aemet.es/es/eltiempo/prediccion/playas"
page = requests.get(url)
tree = html.fromstring(page.text)

In [16]:
dict_playas = dict()
total = 0
url_playa = []

#Saco las provincias de la página
provincias_name = tree.xpath('//select[@id="provincia_selector"]/option/text()')
provincias_id = tree.xpath('//select[@id="provincia_selector"]/option/@value')

#Creo un dataFrame con las provincias y su id para la AEMET
#provincias = pd.DataFrame()
for i in range (0,len(provincias_id)-1):
    #provincias[provincias_id[i+1]] = {"nombre":provincias_name[i+1]}
    url_playas = "http://www.aemet.es/es/eltiempo/prediccion/playas?p="+ provincias_id[i+1]
    page_playa = requests.get(url_playas)
    tree_playa = html.fromstring(page_playa.text)
    poblaciones = tree_playa.xpath('//select[@id="datos_selector"]/optgroup/@label')
    poblaciones_id = tree_playa.xpath('//select[@id="datos_selector"]/optgroup/@value')
    
    for poblacion in poblaciones:
        playas =  tree_playa.xpath('//select[@id="datos_selector"]/optgroup[@label="' + poblacion + '"]/option/text()')
        playas_id = tree_playa.xpath('//select[@id="datos_selector"]/optgroup[@label="' + poblacion + '"]/option/@value')
        for playa in playas_id:
            provincias_id[i+1]
            total+=1

            id_playa = playa [len(playa) - 7 : len(playa)]
            nombre_playa = playa[0:len(playa) - 8 ]
            url = "http://www.aemet.es/es/eltiempo/prediccion/playas/"+playa
            url_playa.append(url)
            texto = '{"_id" :"' + id_playa + '"'
            texto += ',"nombre_playa" : "' + nombre_playa + '" '
            texto += ',"id_largo" : "' + playa + '" '
            texto += ',"nombre_localidad": "' + poblacion + '"'
            texto += ',"nombre_provincia": "' + provincias_name[i+1] + '"'
            texto += ',"id_provincia": "' +provincias_id[i+1] + '"}'
            if id_playa in dict_playas:
                dict_playas[id_playa].append(texto)
            else:
                dict_playas[id_playa]= texto
                
print len(dict_playas)

591


In [17]:
collection = db['costas']
for clave in dict_playas:
    #Convertimos el texto a json para que mongo lo acepte
    d = json.loads(dict_playas[clave])
    post_id = collection.update({u"_id" : clave},{"$set" :d},upsert=True) 

In [18]:
#db.drop_collection("costas")
resultado  = db.costas.find({"nombre_localidad":"Donostia/San Sebastián"})
for doc in resultado:
    print doc

{u'latitud': u'-1.98888888889', u'nombre_playa': u'la-kontxa-la-concha', u'nombre_localidad': u'Donostia/San Sebasti\xe1n', u'2015-07-30 10:20:12': {u'307812': {u'viento_tarde': u'flojo', u'pleamar': u'4:28', u'bajamar': u'4:28', u'temperatua_agua': 22, u'cielo_tarde': u'Nuboso', u'temperatura_maxima': 20, u'viento_manana': u'flojo', u'oleaje_manana': u'd\xe9bil', u'indice_uv': 8, u'oleaje_tarde': u'd\xe9bil', u'sensacion_termica': u'suave', u'cielo_manana': u'Muy nuboso'}}, u'2015-07-31 10:20:12': {u'307812': {u'viento_tarde': u'flojo', u'temperatua_agua': 22, u'cielo_tarde': u'Muy nuboso con lluvia', u'temperatura_maxima': 21, u'viento_manana': u'flojo', u'oleaje_manana': u'd\xe9bil', u'indice_uv': 7, u'oleaje_tarde': u'd\xe9bil', u'sensacion_termica': u'suave', u'cielo_manana': u'Muy nuboso con lluvia'}}, u'id_provincia': u'20', u'nombre_provincia': u'Gipuzkoa', u'2015-07-30 00:00:00': {u'viento_tarde': u'flojo', u'temperatua_agua': 22, u'cielo_tarde': u'Muy nuboso con lluvia', u'te

In [25]:
resultado = db.costas.find({"nombre_provincia": {"$in": ["Cantabria"]}}
                           )
print resultado.count()
for doc in resultado:
    print doc
    break

20
{u'latitud': u'-4.21944444444', u'nombre_playa': u'luana-cobreces', u'2015-07-31 00:00:00': {u'viento_tarde': u'moderado', u'temperatua_agua': 21, u'cielo_tarde': u'Muy nuboso con lluvia', u'temperatura_maxima': 20, u'viento_manana': u'flojo', u'oleaje_manana': u'moderado', u'indice_uv': 7, u'oleaje_tarde': u'moderado', u'sensacion_termica': u'suave', u'cielo_manana': u'Muy nuboso con lluvia'}, u'nombre_localidad': u'Alfoz de Lloredo', u'2015-07-30 10:19:59': {u'558717': {u'viento_tarde': u'flojo', u'pleamar': u'4:20', u'bajamar': u'4:20', u'temperatua_agua': 21, u'cielo_tarde': u'Nuboso', u'temperatura_maxima': 20, u'viento_manana': u'flojo', u'oleaje_manana': u'moderado', u'indice_uv': 8, u'oleaje_tarde': u'moderado', u'sensacion_termica': u'suave', u'cielo_manana': u'Muy nuboso'}}, u'id_provincia': u'39', u'nombre_provincia': u'Cantabria', u'2015-07-30 00:00:00': {u'viento_tarde': u'flojo', u'pleamar': u'4:20', u'bajamar': u'4:20', u'temperatua_agua': 21, u'cielo_tarde': u'Nuboso

In [20]:
#Funciones para convertir las coordenadas
def ConvertDMSToDD(grado, minuto, segundo, direccion):
    dd = float(grado) + float(minuto)/60 + float(segundo)/(60*60)
    if (direccion == "S" or direccion == "O"):
        dd = dd * -1
        # Don't do anything for N or E
    return dd


def ParseDMS(coordenada):
    coordenada = coordenada.strip()
    parts = coordenada.split(" ")
    grado = parts[0].replace("°","_")
    grado = grado[0:len(grado)-1]
    minuto = parts[1][0:len(parts[1])-1]
    segundo = parts [2][0:len(parts[2])-2]
    direccion = parts[3]
    punto = ConvertDMSToDD(grado, minuto, segundo, direccion)
    return punto


print ParseDMS("0° 43' 42'' O")
print ParseDMS("37° 54' 12'' N")

-0.728333333333
37.9033333333


In [23]:
#Empezamos a hacer scrapping de cada una de las playas
playas = dict()
i = 0

print "PLAYAS:"
print "________________________________________________________"
print ""
for url in url_playa:
   
    #url = 'http://www.aemet.es/es/eltiempo/prediccion/playas/xeraco-4614301'
    try:
        id_playa = url[len(url) - 7:len(url) ]

        texto = '{"_id" :"' + id_playa + '"'
        page_playa = requests.get(url)
        tree_playa = html.fromstring(page_playa.text)
        
        div_cabecera = tree_playa.xpath('//*[@class="notas_tabla"]/text()')
        
        poblacion = div_cabecera[3].strip()    
        fecha_completa = div_cabecera[-1]
        fecha = fecha_completa.split(",")
        fecha_convertida = getFechaTime(fecha[1].strip())
        today = datetime.strptime(datetime.today().strftime('%Y-%m-%d'),'%Y-%m-%d')
        if fecha_convertida < today:
            fecha_convertida = today
        #playa =  tree_playa.xpath('//h3[@class="titulo_fondo_azul"]/text()')
   
        coordenadas = tree_playa.xpath('//span[@class="geo"]/abbr/text()')
        

        lati = coordenadas[0].strip().encode('UTF_8')
        longi = coordenadas[1].strip().encode('UTF_8')


        texto += ' , "longitud": "' + str(ParseDMS(lati)) + '" '
        texto += ' , "latitud": "' + str(ParseDMS(longi)) + '" '
        
        #índice uv
        indice_uv_maximo = tree_playa.xpath('//*[starts-with(@class, "raduv_pred")]/text()')
        #cielo
        cielo = tree_playa.xpath('//table[@class="tabla_datos"][1]/tbody/tr[1]/td/img/@title')
        #Viento
        viento = tree_playa.xpath('//table[@class="tabla_datos"][1]/tbody/tr[2]/td/text()')
        #Oleaje
        oleaje = tree_playa.xpath('//table[@class="tabla_datos"][1]/tbody/tr[3]/td/text()')    
        #temperatura máxima
        temperatura_max = tree_playa.xpath('//table[@class="tabla_datos"][1]/tbody/tr[4]/td/div/text()')
        #sensacion tértmica
        sensacion_termica = tree_playa.xpath('//table[@class="tabla_datos"][1]/tbody/tr[5]/td/text()')
        #temperatura agua
        temperatura_agua = tree_playa.xpath('//table[@class="tabla_datos"][1]/tbody/tr[6]/td/text()')
        #pleamar
        hay_tabla2 = False
        if tree_playa.xpath('//table[@class="tabla_datos"][2]/tbody/tr[1]/td/text()'):
            hay_tabla2 = True
            pleamar =  tree_playa.xpath('//table[@class="tabla_datos"][2]/tbody/tr[1]/td/text()')
            #pleamar
            bajamar =  tree_playa.xpath('//table[@class="tabla_datos"][2]/tbody/tr[2]/td/text()')


        texto += ', "' + str(fecha_convertida) + '": {' 
        texto += ' "temperatura_maxima": ' + temperatura_max[0].strip()
        texto += ', "sensacion_termica": "' + sensacion_termica[0].strip() + '" '
        texto += ', "indice_uv": ' + indice_uv_maximo[0].strip()
        texto += ', "temperatua_agua": ' + temperatura_agua[0].strip()
        texto += ', "cielo_manana": "' + cielo[0].strip() + '" '
        texto += ', "cielo_tarde": "' + cielo[1].strip() + '" '
        texto += ', "viento_manana": "' +  viento[0].strip() + '" '
        texto += ', "viento_tarde": "' +  viento[1].strip() + '" '
        texto += ', "oleaje_manana": "' +  oleaje[0].strip() + '" '
        texto += ', "oleaje_tarde": "' +  oleaje[1].strip() + '" '
        if hay_tabla2:
            texto += ', "pleamar": "' + pleamar[0].strip() + '" '
            texto += ', "bajamar": "' + pleamar[0].strip() + '" '
        texto += '}' 

        fecha_mas_1 = fecha_convertida + timedelta(days=1)

        texto += ', "' + str(fecha_mas_1) + '": {'  
        texto += ' "temperatura_maxima": ' + temperatura_max[1].strip()
        texto += ', "sensacion_termica": "' + sensacion_termica[1].strip() + '" '
        texto += ', "indice_uv": ' + indice_uv_maximo[1].strip()
        texto += ', "temperatua_agua": ' + temperatura_agua[1].strip()
        texto += ', "cielo_manana": "' + cielo[2].strip() + '" '
        texto += ', "cielo_tarde": "' + cielo[3].strip() + '" '
        texto += ', "viento_manana": "' +  viento[2].strip() + '" '
        texto += ', "viento_tarde": "' +  viento[3].strip() + '" '
        texto += ', "oleaje_manana": "' +  oleaje[2].strip() + '" '
        texto += ', "oleaje_tarde": "' +  oleaje[3].strip() + '" '
        texto += '}' 
        texto += '}'

        
        d = json.loads(texto)

        post_id = collection.update({u"_id" : id_playa},{"$set" :d},upsert=True) 
    except:
        i+= 1
        print "error:" ,poblacion , " : " , url 

print str(i) , " errores"

PLAYAS:
________________________________________________________

0  errores


In [24]:
today = datetime.strptime(datetime.today().strftime('%Y-%m-%d'),'%Y-%m-%d')
print today

2015-07-30 00:00:00
